## Day 15: Beacon Exclusion Zone

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:

```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
```

In [1]:
import re
import collections
from dataclasses import dataclass

import pytest
from icecream import ic

from vectors import Vector

In [2]:
pat_rule = re.compile(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)")
def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            _match = pat_rule.match(line)
            sensor = Vector(int(_match.group(1)), int(_match.group(2)))
            beacon = Vector(int(_match.group(3)), int(_match.group(4)))               
            yield sensor, beacon

In [3]:
for sensor, beacon in load_input('sample.txt'):
    print(f"Sensor at x={sensor.x}, y={sensor.y}: closest beacon is at x={beacon.x}, y={beacon.y}")

Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3


So, consider the sensor at `2,18`; the closest beacon to it is at `-2,15`. For the sensor at `9,16`, the closest beacon to it is at `10,16`.

Drawing sensors as `S` and beacons as `B`, the above arrangement of sensors and beacons looks like this:

```
               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```

In [4]:
_map = collections.defaultdict(lambda: '.')
for sensor, beacon in load_input('sample.txt'):
    _map[sensor.x, sensor.y] = 'S'
    _map[beacon.x, beacon.y] = 'B'
    
for y in range(-2, 23):
    for x in range(-2, 25):
        print(_map[x, y], end='')
    print()

...........................
...........................
....S......................
......................S....
...............S...........
................SB.........
...........................
...........................
...........................
..........S.......S........
...........................
...........................
....B......................
..S........................
...........................
...........................
..............S.......S....
B..........................
...........SB..............
................S..........
....S......................
...........................
............S......S.......
...........................
.......................B...


This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

```
               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```

This sensor's closest beacon is at `2,10`, and so you know there are no beacons that close or closer (in any positions marked #).

In [5]:
def at_distance(v: Vector, d: int) -> [Vector]:
    if d == 0:
        return []
    elif d == 1:
        return [Vector(v.x, v.y-1), Vector(v.x+1, v.y), Vector(v.x, v.y+1), Vector(v.x - 1, v.y)]
    result = []
    for delta_y in range(-d, d+1):
        width = d - abs(delta_y)
        for delta_x in range(-width, width+1):
            result.append(Vector(v.x + delta_x, v.y + delta_y))
    return result

In [6]:
assert at_distance(Vector(2, 3), d=0) == []
assert at_distance(Vector(2, 3), d=1) == [Vector(2, 2), Vector(3, 3), Vector(2, 4), Vector(1, 3)]
assert at_distance(Vector(2, 3), d=2) == [
    Vector(2, 1),
    Vector(1, 2), Vector(2, 2), Vector(3, 2),
    Vector(0, 3), Vector(1, 3), Vector(2, 3), Vector(3, 3), Vector(4, 3),
    Vector(1, 4), Vector(2, 4), Vector(3, 4), 
    Vector(2, 5),
    ]


In [7]:
_map = collections.defaultdict(lambda: '.')
for sensor, beacon in load_input('sample.txt'):
    _map[sensor.x, sensor.y] = 'S'
    _map[beacon.x, beacon.y] = 'B'
    
for v in at_distance(Vector(8, 7), d=9):
    if _map[v.x, v.y] == '.':
        _map[v.x, v.y] = '#'
        
for y in range(-2, 23):
    for x in range(-2, 25):
        print(_map[x, y], end='')
    print()

..........#................
.........###...............
....S...#####..............
.......#######........S....
......#########S...........
.....###########SB.........
....#############..........
...###############.........
..#################........
.#########S#######S#.......
..#################........
...###############.........
....B############..........
..S..###########...........
......#########............
.......#######.............
........#####.S.......S....
B........###...............
..........#SB..............
................S..........
....S......................
...........................
............S......S.......
...........................
.......................B...


None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

```
                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.
```

In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

In [8]:
def manhattan(v0: Vector, v1: Vector) -> int:
    return abs(v1.x - v0.x) + abs(v1.y - v0.y)

assert manhattan(Vector(0, 0), Vector(2, 2)) == 4
assert manhattan(Vector(0, 0), Vector(-2, -2)) == 4
assert manhattan(Vector(0, 0), Vector(-2, 3)) == 5
assert manhattan(Vector(0, 0), Vector(2, -3)) == 5
assert manhattan(Vector(-1, 0), Vector(2, -3)) == 6

In [9]:
_map = collections.defaultdict(lambda: '.')
for sensor, beacon in load_input('sample.txt'):
    _map[sensor.x, sensor.y] = 'S'
    _map[beacon.x, beacon.y] = 'B'
    d = manhattan(sensor, beacon)
    for v in at_distance(sensor, d):
        if _map[v.x, v.y] == '.':
            _map[v.x, v.y] = '#'

min_x = min([t[0] for t in _map])
max_x = max([t[0] for t in _map])
min_y = min([t[1] for t in _map])
max_y = max([t[1] for t in _map])
       
for y in range(min_y, max_y+1):
    print(f'{y:7}', end=' ')
    for x in range(min_x, max_x+1):
        print(_map[x, y], end='')
    print()
y = 10
count = 0
for x in range(min_x, max_x+1):
    print(_map[x, y], end='')
    if _map[x, y] == '#':
        count += 1

print(count) 

    -10 ..........#..........................
     -9 .........###.........................
     -8 ........#####........................
     -7 .......#######.......................
     -6 ......#########.............#........
     -5 .....###########...........###.......
     -4 ....#############.........#####......
     -3 ...###############.......#######.....
     -2 ..#################.....#########....
     -1 .###################.#.###########...
      0 ##########S########################..
      1 .###########################S#######.
      2 ..###################S#############..
      3 ...###################SB##########...
      4 ....#############################....
      5 .....###########################.....
      6 ......#########################......
      7 .......#########S#######S#####.......
      8 ........#######################......
      9 .......#########################.....
     10 ......####B######################....
     11 .....###S#############.###

In [10]:
def fill_places(sensor, beacon, line):
    d = manhattan(sensor, beacon)
    delta = abs(sensor.y - line)
    if delta > d:
        return []
    # ic(line, d, delta, (d - delta), abs(d-delta), abs(d-delta)*2+1)
    length = d - delta
    return list(range(sensor.x-length, sensor.x+length+1))

sensor = Vector(3, 3)
beacon = Vector(7, 7)
assert fill_places(sensor, beacon, -6) == []
assert fill_places(sensor, beacon, -5) == [3]
assert fill_places(sensor, beacon, -4) == [2, 3, 4]
assert fill_places(sensor, beacon, -3) == [1, 2, 3, 4, 5]
assert fill_places(sensor, beacon, -2) == [0, 1, 2, 3, 4, 5, 6]
assert fill_places(sensor, beacon, -1) == [-1, 0, 1, 2, 3, 4, 5, 6, 7]
assert fill_places(sensor, beacon, 0) == [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
assert fill_places(sensor, beacon, 1) == [-3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
assert fill_places(sensor, beacon, 2) == [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
assert fill_places(sensor, beacon, 3) == [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
assert fill_places(sensor, beacon, 4) == [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
assert fill_places(sensor, beacon, 5) == [-3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
assert fill_places(sensor, beacon, 6) == [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
assert fill_places(sensor, beacon, 7) == [-1, 0, 1, 2, 3, 4, 5, 6, 7]
assert fill_places(sensor, beacon, 8) == [0, 1, 2, 3, 4, 5, 6]
assert fill_places(sensor, beacon, 9) == [1, 2, 3, 4, 5]
assert fill_places(sensor, beacon, 10) == [2, 3, 4]
assert fill_places(sensor, beacon, 11) == [3]
assert fill_places(sensor, beacon, 12) == []

In [11]:
def solution_one(filename, line, tron=False):
    result_set = set([])
    for sensor, beacon in load_input(filename):
        d = manhattan(sensor, beacon)
        if (sensor.y - d) <= line < (sensor.y + d):
            if tron:
                print(f"sensor {sensor}, beacon {beacon} is valid", end=' ')
                print(f"distance: {d} range: {sensor.y - d}:{sensor.y + d + 1}", end=' ')
                print(f"places: {len(fill_places(sensor, beacon, line))}")
            result_set.update(fill_places(sensor, beacon, line))
    return len(result_set) - 1 # The beacon itself

assert solution_one('sample.txt', 9) == 24
assert solution_one('sample.txt', 10) == 26
assert solution_one('sample.txt', 11) == 27

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

In [12]:
sol = solution_one('input.txt', 2000000)
print(f"Solution part one: {sol-1}")

Solution part one: 4737566


## Part Two

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

In [13]:
_map = collections.defaultdict(lambda: '.')
for sensor, beacon in load_input('sample.txt'):
    _map[sensor.x, sensor.y] = 'S'
    _map[beacon.x, beacon.y] = 'B'
    d = manhattan(sensor, beacon)
    for v in at_distance(sensor, d):
        if _map[v.x, v.y] == '.':
            _map[v.x, v.y] = '#'
       
for y in range(0, 21):
    print(f'{y:7}', end=' ')
    for x in range(0, 21):
        print(_map[x, y], end='')
    print()

      0 ##S##################
      1 ####################S
      2 #############S#######
      3 ##############SB#####
      4 #####################
      5 #####################
      6 #####################
      7 ########S#######S####
      8 #####################
      9 #####################
     10 ##B##################
     11 S#############.######
     12 #####################
     13 #####################
     14 ############S#######S
     15 #####################
     16 #########SB##########
     17 ##############S######
     18 ##S##################
     19 #####################
     20 ##########S######S###


Find the only possible position for the distress beacon. What is its tuning frequency?

> Necesitamos una estructura para almacenar rangos

In [311]:
@dataclass
class Lapse:
    left: int
    right: int
    
    def __post_init__(self):
        if self.left > self.right:
            self.left, self.right = self.right, self.left
    
    def __eq__(self, other):
        return self.left == other.left and self.right == other.right
    
    def __lt__(self, other):
        return self.left < other.left

    def __le__(self, other):
        return self.left <= other.left
    
    def __str__(self):
        return f"{self.left}~{self.right}"
    
    def overlaps(self, other: Lapse) -> bool:
        if self > other:
            raise ValueError("Lap2 most be greater or equal than l1")    
        return other.left <= (self.right + 1)
    
assert Lapse(2, 3) == Lapse(2, 3)
assert Lapse(2, 3) < Lapse(3, 3)
assert Lapse(2, 3) > Lapse(1, 3)

assert Lapse(2, 5).overlaps(Lapse(7, 9)) is False
assert Lapse(-4, 4).overlaps(Lapse(5, 2)) is True
assert Lapse(2, 5).overlaps(Lapse(5, 6)) is True
assert Lapse(5, 6).overlaps(Lapse(8, 26)) is False
with pytest.raises(ValueError):
    Lapse(7, 9).overlaps(Lapse(2, 5))

In [312]:
import random

laps = sorted([
    Lapse(left=random.randrange(-20, 21), right=random.randrange(-20, 21))
    for _ in range(5)
])
laps

[Lapse(left=-19, right=15),
 Lapse(left=-14, right=-10),
 Lapse(left=-9, right=-2),
 Lapse(left=-7, right=3),
 Lapse(left=9, right=14)]

In [378]:
def compress(laps: [Lapse]) -> [Lapse]:
    if len(laps) < 2:
        return laps
    prev = laps.pop(0)
    result = [prev]
    index = 0
    while laps:
        other = laps.pop(0)
        while prev.overlaps(other):
            result[index] = prev = Lapse(prev.left, max(prev.right, other.right))
            if laps:
                other = laps.pop(0)
            else:
                other = None
                break
        if other:
            result.append(other)
        prev = other
        index += 1
    return result

In [379]:
compress([
    Lapse(left=0, right=6),
    Lapse(left=9, right=9),
    Lapse(left=9, right=15),
])

[Lapse(left=0, right=6), Lapse(left=9, right=15)]

In [380]:
assert compress([Lapse(2, 2), Lapse(left=15, right=18)]) == [Lapse(left=2, right=2), Lapse(left=15, right=18)]

In [381]:
assert compress([Lapse(left=3, right=7), Lapse(left=7, right=14)]) == [Lapse(left=3, right=14)]

In [382]:
assert compress([Lapse(left=15, right=25), Lapse(left=15, right=17)]) == [Lapse(left=15, right=25)]

In [384]:
assert compress([Lapse(left=5, right=27), Lapse(left=5, right=15)]) == [Lapse(left=5, right=27)]

In [386]:
assert compress([Lapse(left=9, right=15), Lapse(left=9, right=9)]) == [Lapse(left=9, right=15)]

In [387]:
laps = [
    Lapse(left=-15, right=7),
    Lapse(left=-5, right=3),
    Lapse(left=-4, right=17),
    Lapse(left=6, right=11),
    Lapse(left=12, right=14)
]
assert compress(laps) == [Lapse(left=-15, right=17)]

In [409]:
def solution_two(filename, scan_range):
    all_x = [list() for _ in range(scan_range)]
    all_y = [list() for _ in range(scan_range)]
    count = 0
    for sensor, beacon in load_input(filename):
        d = manhattan(sensor, beacon)
        for delta_x in range(-d, d+1):
            x = sensor.x + delta_x
            if 0 <= x < scan_range:
                h = d - abs(delta_x)
                all_x[x].append(
                    Lapse(max(0, sensor.y - h), min(scan_range, sensor.y + h))
                )
                all_x[x] = compress(sorted(all_x[x]))
        for delta_y in range(-d, d + 1):
            y = sensor.y + delta_y
            if 0 <= y < scan_range:
                h = d - abs(delta_y)
                all_y[y].append(
                    Lapse(max(0, sensor.x - h), min(scan_range, sensor.x + h))
                )
                all_y[y] = compress(sorted(all_y[y]))

    for index in range(scan_range):
        if len(all_x[index]) > 1:
            y = all_x[index][0].right + 1
            break
    for index in range(scan_range):
        if len(all_y[index]) > 1:
            x = all_y[index][0].right + 1
            break
    freq = x * 4000000 + y
    return freq

assert solution_two('sample.txt', 20) == 56000011

In [413]:
sol = solution_two('input.txt', 4000000)
print(f"Solution for part two: {sol}")

Solution for part two: 13267474686239
